In [7]:
import pandas as pd
import pandas_datareader as pdr
import requests
import zipfile
import io

In [2]:
lista_datas = [201501, 201502, 201503, 201504, 201601, 201602, 201603, 201604, 201701, 201702, 201703, 201704, 201801, 201802, 201803, 201804, 201901, 201902, 201903, 201904, 202001, 202002]

In [3]:
def buscar_relatorio(nome_bucket, conta, ano, trimestre):
    try:
        
        if conta.upper() == 'ATIVO' or conta.upper() == 'PASSIVO' or conta.upper() == 'DRE':
        
            # Link do CSV
            url = 'https://{}.s3-sa-east-1.amazonaws.com/{}/{}0{}.csv'.format(nome_bucket, conta.title(), ano, trimestre)

            # Função para ler o CSV
            df = pd.read_csv(url, sep=";")


            if conta.upper() == 'ATIVO':

                # Renomeação das colunas
                df = df.rename(
                columns={
                    'Disponibilidades (a)': 'Disponibilidades', 
                    'Aplicações Interfinanceiras de Liquidez (b)': 'Aplicações Interfinanceiras de Liquidez',
                    'TVM e Instrumentos Financeiros Derivativos (c)': 'TVM e Instrumentos Financeiros Derivativos',
                    'Operações de Crédito (d1)': 'Operações de Crédito',
                    'Unnamed: 13': 'Provisão sobre Operações de Crédito',
                    'Unnamed: 14': 'Operações de Crédito Líquidas de Provisão',
                    'Arrendamento Mercantil': 'Arrendamento Mercantil a Receber',
                    'Unnamed: 16': 'Imobilizado de Arrendamento',
                    'Unnamed: 17': 'Credores por Antecipação de Valor Residual',
                    'Unnamed: 18': 'Provisão sobre Arrendamento Mercantil',
                    'Provisão sobre Arrendamento Mercantil (e4)': 'Provisão sobre Arrendamento Mercantil',
                    'Unnamed: 19': 'Arrendamento Mercantil Líquido de Provisão',
                    'Outros Créditos - Líquido de Provisão (f)': 'Outros Créditos - Líquido de Provisão',
                    'Outros Ativos Realizáveis (g)': 'Outros Ativos Realizáveis',
                    'Permanente Ajustado (h)': 'Permanente Ajustado',
                    'Ativo Total Ajustado (i) = (a) + (b) + (c) + (d) + (e) + (f) + (g) + (h)': 'Ativo Total Ajustado',
                    'Credores por Antecipação de Valor Residual (j)': 'Credores por Antecipação de Valor Residual',
                    'Ativo Total (k) = (i) - (j)': 'Ativo Total' 
                })

                # Exclusão da coluna vazia
                df = df.drop(columns='Unnamed: 26')

                # Excluí os dados da primeira linha
                df = df.drop([0])

                # Encontra o índice do fim dos dados das entidades financeiras
                for i in range(len(df)):
                    if df.iloc[i][0] == 'TCB - Tipo de Consolidado Bancário':
                        marcador = i

                # Excluí os dados
                df = df.drop(df.index[marcador:len(df)])

                # Reseta os índices para posiciona-los de maneira correta
                df = df.reset_index(drop=True)

                # Retorna o dataframe como resultado da função
                return df       

            elif conta.upper() == 'PASSIVO':

                # Renomeação das colunas
                df = df.rename(
                columns={
                    'Captações': 'Depósitos à Vista',
                    'Unnamed: 10': 'Depósitos de Poupança',
                    'Unnamed: 11': 'Depósitos Interfinanceiros',
                    'Unnamed: 12': 'Depósitos a Prazo',
                    'Unnamed: 13': 'Outros Depósitos',
                    'Unnamed: 14': 'Depósito Total',
                    'Unnamed: 15': 'Obrigações por Operações Compromissadas',
                    'Unnamed: 16': 'Letras de Crédito Imobiliário',
                    'Unnamed: 17': 'Letras de Crédito do Agronegócio',
                    'Unnamed: 18': 'Letras Financeiras',
                    'Unnamed: 19': 'Obrigações por Títulos e Valores Mobiliários no Exterior',
                    'Unnamed: 20': 'Outros Recursos de Aceites e Emissão de Títulos',
                    'Unnamed: 21': 'Recursos de Aceites e Emissão de Títulos',
                    'Unnamed: 22': 'Obrigações por Empréstimos e Repasses',
                    'Unnamed: 23': 'Captações',
                    'Instrumentos Derivativos (f)': 'Instrumentos Derivativos',
                    'Outras Obrigações (g)': 'Outras Obrigações',
                    'Passivo Circulante e Exigível a Longo Prazo (h) = (e) + (f) + (g)': 'Passivo Circulante e Exigível a Longo Prazo',
                    'Resultados de Exercícios Futuros (i)': 'Resultados de Exercícios Futuros',
                    'Patrimônio Líquido (j)': 'Patrimônio Líquido',
                    'Passivo Total (k) = (h) + (i) + (j)': 'Passivo Total'
                })

                # Exclusão da coluna vazia
                df = df.drop(columns='Unnamed: 30')

                # Excluí os dados da primeira linha e segunda linha
                df = df.drop([0, 1])

                # Encontra o índice do fim dos dados das entidades financeiras
                for i in range(len(df)):
                                if df.iloc[i][0] == 'TCB - Tipo de Consolidado Bancário':
                                    marcador = i

                # Excluí os dados
                df = df.drop(df.index[marcador:len(df)])

                # Reseta os índices para posiciona-los de maneira correta
                df = df.reset_index(drop=True)

                # Retorna o dataframe como resultado da função
                return df

            elif conta.upper() == 'DRE':

                # Renomeação das colunas
                df = df.rename(
                columns={
                    'Resultado de Intermediação Financeira': 'Rendas de Operações de Crédito',
                    'Unnamed: 10': 'Rendas de Operações de Arrendamento Mercantil',
                    'Unnamed: 11': 'Rendas de Operações com TVM',
                    'Unnamed: 12': 'Rendas de Operações com Instrumentos Financeiros Derivativos',
                    'Unnamed: 13': 'Resultado de Operações de Câmbio',
                    'Unnamed: 14': 'Rendas de Aplicações Compulsórias',
                    'Unnamed: 15': 'Receitas de Intermediação Financeira',
                    'Unnamed: 16': 'Despesas de Captação',
                    'Unnamed: 17': 'Despesas de Obrigações por Empréstimos e Repasses',
                    'Unnamed: 18': 'Despesas de Operações de Arrendamento Mercantil',
                    'Unnamed: 19': 'Resultado de Operações de Câmbio',
                    'Unnamed: 20': 'Resultado de Provisão para Créditos de Difícil Liquidação',
                    'Unnamed: 21': 'Despesas de Intermediação Financeira',
                    'Unnamed: 22': 'Resultado de Intermediação Financeira',
                    'Outras Receitas/Despesas Operacionais': 'Rendas de Prestação de Serviços',
                    'Unnamed: 24': 'Rendas de Tarifas Bancárias',
                    'Unnamed: 25': 'Despesas de Pessoal',
                    'Unnamed: 26': 'Despesas Administrativas',
                    'Unnamed: 27': 'Despesas Tributárias',
                    'Unnamed: 28': 'Resultado de Participações',
                    'Unnamed: 29': 'Outras Receitas Operacionais',
                    'Unnamed: 30': 'Outras Despesas Operacionais',
                    'Unnamed: 31': 'Outras Receitas/Despesas  Operacionais',
                    'Resultado Operacional (e) = (c) + (d)': 'Resultado Operacional',
                    'Resultado Não Operacional (f)': 'Resultado Não Operacional',
                    'Resultado antes da Tributação, Lucro e Participação (g) = (e) + (f)': 'Resultado antes da Tributação, Lucro e Participação',
                    'Imposto de Renda e Contribuição Social (h)': 'Imposto de Renda e Contribuição Social',
                    'Participação nos Lucros (i)': 'Participação nos Lucros',
                    'Lucro Líquido (j) = (g) + (h) + (i)': 'Lucro Líquido',
                    'Juros Sobre Capital Social de Cooperativas (k)': 'Juros Sobre Capital Social de Cooperativas'
                })

                # Exclusão da coluna vazia
                df = df.drop(columns='Unnamed: 39')

                # Excluí os dados da primeira linha e segunda linha
                df = df.drop([0, 1])

                # Encontra o índice do fim dos dados das entidades financeiras
                for i in range(len(df)):
                                if df.iloc[i][0] == 'TCB - Tipo de Consolidado Bancário':
                                    marcador = i

                # Excluí os dados
                df = df.drop(df.index[marcador:len(df)])

                # Reseta os índices para posiciona-los de maneira correta
                df = df.reset_index(drop=True)

                # Retorna o dataframe como resultado da função
                return df

        else:
            print('Nome da conta inserido incorretamente. Tente novamente passando um dos possíveis valores: "ativo", "passivo" ou "DRE".')
    except:
        # Tratamento de erro
        return print('Arquivo não encontrado. Possível motivo: \n'
                        '- Período procurado inexistente. \n'
                        '- Biblioteca "Pandas" não importada. \n'
                    )

In [4]:
def buscar_relatorios(nome_bucket, conta, ano_inicial, trimestre_inicial, ano_final, trimestre_final):
    nr_relatorios = (ano_final - ano_inicial) * 4 + (trimestre_final - trimestre_inicial) + 1
    for i in lista_datas:
        if i == (ano_inicial * 100) + trimestre_inicial:
            posicao = lista_datas.index(i)
            break
    
    for j in range(nr_relatorios):
        if j == 0:
            dado = buscar_relatorio(nome_bucket, conta, int(str(lista_datas[posicao + j])[:4]), int(str(lista_datas[posicao + j])[-1:]))
            dados = dado
        else:
            dado = buscar_relatorio(nome_bucket, conta, int(str(lista_datas[posicao + j])[:4]), int(str(lista_datas[posicao + j])[-1:]))
            dados = dados.append(dado)
        
    return dados

In [5]:
def periodos_disponiveis():
    print('Dados disponíveis: \n' +
         str(min(lista_datas))[-1:] + 
         'º Trimestre/' +
         str(min(lista_datas))[:4] + 
         ' à ' +
         str(max(lista_datas))[-1:] +
         'º Trimestre/' +
         str(max(lista_datas))[:4] +
         '.'
         )

In [6]:
def ultimo_relatorio_disponivel(conta):
    print('\033[1m2º Trimestre/2020\033[0m')
    return buscar_relatorio(conta, int(str(max(lista_datas))[:4]), int(str(max(lista_datas))[-1:]))

In [9]:
def buscar_balancetes(safra_inicial, safra_final):

    qtd_meses = ((int(safra_final[:4]) - int(safra_inicial[:4])) * 12) + (int(safra_final[-2:]) - int(safra_inicial[-2:])) + 1
    lista_datas = []
    i = 0

    for mes in range(qtd_meses):
        if i == 0 and int(safra_inicial[-2:]) == 12:
            data = str(((int(safra_inicial[:4]) + 1) * 100 + 1))
        elif i == 0 and int(safra_inicial[-2:]) != 12:
            data = safra_inicial
        elif i > 0 and int(data[-2:]) != 12:
            data = str(int(data) + 1)
        else:
            data = str(((int(data[:4]) + 1) * 100 + 1))
        lista_datas.append(data)
        i = 1
        
    appended_data = []

    for dt in lista_datas:
        url = 'http://www4.bcb.gov.br/fis/cosif/cont/balan/bancos/'+ dt + 'BANCOS.ZIP'
        r = requests.get(url)
        z = zipfile.ZipFile(io.BytesIO(r.content))
        unzipped_file = z
        df = pd.read_csv(unzipped_file.open(zipfile.ZipFile.namelist(unzipped_file)[0]), sep=';', engine='python', skiprows=3)
        appended_data.append(df)

    appended_data = pd.concat(appended_data)
    return appended_data

In [10]:
def dados_acoes_yahoo(tickers, data_inicial, data_final, tipo):
    
    if tipo.upper() == 'G':
        dados = wb.DataReader(tickers, data_source='yahoo', start = data_inicial, end = data_final)
        dados_finais = dados.stack()
    elif tipo.upper() == 'A':
        dados = wb.DataReader(tickers, data_source='yahoo', start = data_inicial, end = data_final)['Adj Close']
        dados_finais = dados.stack()
    elif tipo.upper() == 'C':
        dados = wb.DataReader(tickers, data_source='yahoo', start = data_inicial, end = data_final)['Close']
        dados_finais = dados.stack()
    elif tipo.upper() == 'H':
        dados = wb.DataReader(tickers, data_source='yahoo', start = data_inicial, end = data_final)['High']
        dados_finais = dados.stack()
    elif tipo.upper() == 'L':
        dados = wb.DataReader(tickers, data_source='yahoo', start = data_inicial, end = data_final)['Low']
        dados_finais = dados.stack()
    elif tipo.upper() == 'O':
        dados = wb.DataReader(tickers, data_source='yahoo', start = data_inicial, end = data_final)['Open']
        dados_finais = dados.stack()
    elif tipo.upper() == 'V':
        dados = wb.DataReader(tickers, data_source='yahoo', start = data_inicial, end = data_final)['Volume']
        dados_finais = dados.stack()
    else:
        return print('Tipo não encontrado. Tente novamente com um dos possíveis parâmetros ("G", "F", "C", "H", "L", "O" ou "V")')
            
    return dados_finais

In [11]:
def plotar_grafico_fechamentos(tickers, data_inicial, data_final):
    dados = wb.DataReader(tickers, data_source='yahoo', start = data_inicial, end = data_final)['Adj Close']
    dados_finais = dados.div(dados.iloc[0]/100.0)
    dados_finais.plot(figsize=(16,8))

In [13]:
def retornos_simples(tickers, data_inicial, data_final):
    returns = pd.DataFrame()
    for ticker in tickers:
        dados = wb.DataReader(ticker, data_source='yahoo', start = data_inicial, end = data_final)
        returns[ticker] = (dados['Adj Close']/dados['Adj Close'].shift(1)) - 1
        
    return returns

In [14]:
def retornos_log(tickers, data_inicial, data_final):
    returns = pd.DataFrame()
    for ticker in tickers:
        dados = wb.DataReader(ticker, data_source='yahoo', start = data_inicial, end = data_final)
        returns[ticker] = np.log(dados['Adj Close'])/np.log(dados['Adj Close'].shift(1))
        
    return returns

In [27]:
def ajuda():
    print('----------------------\033[1m Comandos \033[0m------------------------ \n'
          '\n'
          '---------------------- Relatórios da AWS ----------------------'
          '\n'
          '\n'
          'buscar_relatorio(conta, ano, trimestre): \n'
          '- Objetivo: busca os dados das demonstrações financeiras de conglomerados financeiros. \n' 
          '- Primeiro Parâmetro (conta): informar o relatório desejado (ativo, passivo, DRE). \n'
          '- Segundo Parâmetro (ano): informar o ano desejado. \n'
          '- Terceiro Parâmetro (trimestre): informar o trimestre desejado. \n'
          '\n'
          'buscar_relatorios(conta, ano_inicial, trimestre_inicial, ano_final, trimestre_final): \n'
          '- Objetivo: busca os dados das demonstrações financeiras de conglomerados financeiros no intervalo determinado. \n' 
          '- Primeiro Parâmetro (conta): informar o relatório desejado (ativo, passivo, DRE). \n'
          '- Segundo Parâmetro (ano_inicial): informar o ano inicial. \n'
          '- Terceiro Parâmetro (trimestre_inicial): informar o trimestre inicial desejado. \n'
          '- Quarto Parâmetro (ano_final): informar o ano final. \n'
          '- Quinto Parâmetro (trimestre_final): informar o trimestre final desejado. \n'
          '\n'
          'ultimo_relatorio_disponivel(conta): \n'
          '- Objetivo: traz o relatório disponível mais recente. \n'
          '- Parâmetro (conta): informar o relatório desejado (ativo, passivo, DRE).\n'
          '\n'
          '---------------------- Relatórios Web Scraping ----------------------'
          '\n'
          '\n'
          'buscar_balancetes(safra_inicial, safra_final): \n'
          '- Objetivo: busca os dados dos balancetes divulgados pelo Bacen das Instituições Financeiras. \n'
          '- Primeiro Parâmetro: informar a safra inicial desejada no formato anomês, ex: 202001. \n'
          '- Segundo Parâmetro: informar a safra final desejada no formato anomês, ex: 202004. \n'
          '\n'
          '---------------------- Relatórios Yahoo Finance ----------------------'
          '\n'
          '\n'
          'dados_acoes_yahoo(tickers, data_inicial, data_final, tipo): \n'
          '- Objetivo: traz em formato de tabela o histórico das ações do tipo selecionado. \n'
          '- Primeiro Parâmetro: código da ação. Exemplo: MGLU3.SA. \n'
          '- Segundo Parâmetro: data inicial no formato "ano-mes-dia", ex: "2020-01-01". \n'
          '- Terceiro Parâmetro: data final no formato "ano-mes-dia", ex: "2020-02-10". \n'
          '- Quarto Parâmetro: tipo do relatório ("G", "F", "C", "H", "L", "O" ou "V"), no qual: \n'
          '        G: Todas as informações disponíveis \n'
          '        F: Preço de Fechamentos Ajustados \n'
          '        C: Preços de Fechamento \n'
          '        H: Preços Máximos \n'
          '        L: Preços Mínimos \n'
          '        O: Preços de Abertura \n'
          '        V: Volume de Negociações \n'
          '\n'
          'plotar_grafico_fechamentos(tickers, data_inicial, data_final): \n'
          '- Objetivo: plota o gráfico de fechamento das ações selecionadas na base 100. \n'
          '- Primeiro Parâmetro: código da ação. Exemplo: MGLU3.SA. \n'
          '- Segundo Parâmetro: data inicial no formato "ano-mes-dia", ex: "2020-01-01". \n'
          '- Terceiro Parâmetro: data final no formato "ano-mes-dia", ex: "2020-02-10". \n'
          '\n'
          'retornos_simples(tickers, data_inicial, data_final): \n'
          '- Objetivo: traz em formato de tabela os retornos simples das ações selecionadas. \n'
          '- Primeiro Parâmetro: código da ação. Exemplo: MGLU3.SA. \n'
          '- Segundo Parâmetro: data inicial no formato "ano-mes-dia", ex: "2020-01-01". \n'
          '- Terceiro Parâmetro: data final no formato "ano-mes-dia", ex: "2020-02-10". \n'
          '\n'
          'retornos_log(tickers, data_inicial, data_final): \n'
          '- Objetivo: traz em formato de tabela os retornos logarítmicos das ações selecionadas. \n'
          '- Primeiro Parâmetro: código da ação. Exemplo: MGLU3.SA. \n'
          '- Segundo Parâmetro: data inicial no formato "ano-mes-dia", ex: "2020-01-01". \n'
          '- Terceiro Parâmetro: data final no formato "ano-mes-dia", ex: "2020-02-10". \n'
          '\n'
          '---------------------- Auxiliares ------------------------'
          '\n'
          '\n'
          'ajuda(): \n'
          '- Objetivo: mostra todos os comandandos e como utiliza-los. \n'
          '\n'
          'periodos_disponiveis(): \n'
          '- Objetivo: mostra todos os períodos de relatórios disponíveis até o momento. \n'
          '\n'
          '----------------------\033[1m Contato \033[0m------------------------ \n'
          '\n'
          'Para mais informações, dúvidas ou sugestões entre em contato com: \n'
          '- fabio_bragato@hotmail.com \n'
          '- fabio.bragato.carmo@usp.br'
         )

In [28]:
ajuda()

---------------------- Comandos ------------------------ 

---------------------- Relatórios da AWS ----------------------

buscar_relatorio(conta, ano, trimestre): 
- Objetivo: busca os dados das demonstrações financeiras de conglomerados financeiros. 
- Primeiro Parâmetro (conta): informar o relatório desejado (ativo, passivo, DRE). 
- Segundo Parâmetro (ano): informar o ano desejado. 
- Terceiro Parâmetro (trimestre): informar o trimestre desejado. 

buscar_relatorios(conta, ano_inicial, trimestre_inicial, ano_final, trimestre_final): 
- Objetivo: busca os dados das demonstrações financeiras de conglomerados financeiros no intervalo determinado. 
- Primeiro Parâmetro (conta): informar o relatório desejado (ativo, passivo, DRE). 
- Segundo Parâmetro (ano_inicial): informar o ano inicial. 
- Terceiro Parâmetro (trimestre_inicial): informar o trimestre inicial desejado. 
- Quarto Parâmetro (ano_final): informar o ano final. 
- Quinto Parâmetro (trimestre_final): informar o trimestre fi